<a href="https://colab.research.google.com/github/Shai2u/demographic_estimation_dashboard_article/blob/main/population_simulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Installing specific packages.
!pip install dash==2.0.0
!pip install jupyter_dash
!pip install geopandas
!pip install dash-leaflet==0.1.23
!pip install dash-extensions==0.0.65

     |████████████████████████████████| 7.3 MB 4.2 MB/s 
     |████████████████████████████████| 357 kB 29.7 MB/s 
  Created wheel for retrying: filename=retrying-1.3.3-py3-none-any.whl size=11447 sha256=adb707e74da2528ebd07f31ee23eaebf954344f6e07d2c943303685e26a09a04
  Stored in directory: /root/.cache/pip/wheels/f9/8d/8d/f6af3f7f9eea3553bc2fe6d53e4b287dad18b06a861ac56ddf
Successfully built retrying
     |████████████████████████████████| 1.0 MB 5.5 MB/s 
     |████████████████████████████████| 16.7 MB 212 kB/s 
     |████████████████████████████████| 6.3 MB 38.1 MB/s 
     |████████████████████████████████| 1.9 MB 5.2 MB/s 
     |████████████████████████████████| 1.8 MB 5.3 MB/s 
     |████████████████████████████████| 73 kB 1.5 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for jsbeautifier: filename=jsbeautifier-1.14.2-py3-none-any.whl size=92134 sha256=4a855298fa4c23988d4b569b14f76

In [5]:

from dash import Dash, html, dcc, Input, Output
from dash_extensions.javascript import  assign
from plotly.subplots import make_subplots

import dash_leaflet.express as dlx
import plotly.express as px
import plotly.graph_objects as go
import dash_leaflet as dl
import pandas as pd
import numpy as np
import json
import geopandas as gpd
from jupyter_dash import JupyterDash
import datetime

In [6]:
#pandas configuration
pd.options.display.max_columns=100

# Loading Files

In [7]:

#Statistical Stats
statisticalStatsGdf = gpd.read_file('https://raw.githubusercontent.com/Shai2u/demographic_estimation_dashboard_article/main/dashboard/data/statistical_tract_4326.geojson')
statisticalStatsJson = json.loads(statisticalStatsGdf.to_json())
#building simulation
simulatedBldgsGdf = gpd.read_file('https://raw.githubusercontent.com/Shai2u/demographic_estimation_dashboard_article/main/dashboard/data/buildings_for_dashboard_4326.geojson')
simulatedBldgsJson = json.loads(simulatedBldgsGdf.to_json())
simulatedBldgsGdf['start_date'] = pd.to_datetime(simulatedBldgsGdf['start_date'])
simulatedBldgsGdf['end_date'] = pd.to_datetime(simulatedBldgsGdf['end_date'])
#agents track
agents_track_status = pd.read_csv('https://raw.githubusercontent.com/Shai2u/demographic_estimation_dashboard_article/main/dashboard/data/agents_track_status.csv')
agents_track_status.drop(columns='Unnamed: 0',inplace=True)

agents_stat_summary_by_year = pd.read_csv('https://raw.githubusercontent.com/Shai2u/demographic_estimation_dashboard_article/main/dashboard/data/yearly_stats_for_dashboard.csv')

year_ranges = agents_stat_summary_by_year['year']

color_labels = pd.read_excel(
    'https://raw.githubusercontent.com/Shai2u/demographic_estimation_dashboard_article/main/assets/color_labels.xlsx')
colorDict = dict(zip(color_labels['label'], color_labels['colors_']))


attribution = '© OpenStreetMap contributors, © CARTO'
cartoUrl = 'http://basemaps.cartocdn.com/light_all/{z}/{x}/{y}.png'

# style =   color: 'red',
#                 weight: 10,
#                 opacity: .7,
#                 dashArray: '20,15',
#                 lineJoin: 'round'


# Agents Non Agregated preprocessing

In [8]:
incomeDict = {9000:'Low',19500:'Medium', 1000000:'High'}
agents_track_status['income_cat'] = pd.cut(agents_track_status['income'], [0,9000,19500,1000000],right=True, labels=['Low','Medium','High'],ordered=True)

agents_track_status.loc[agents_track_status['rent']==1,'rent_own']='Rent'
agents_track_status.loc[agents_track_status['rent']==0,'rent_own']='Own'

## Pre-processing

In [9]:

agents_stat_summary_by_year['New Comers'] = agents_stat_summary_by_year['New Comers_income_low'] + agents_stat_summary_by_year['New Comers_income_medium'] +  + agents_stat_summary_by_year['New Comers_income_high']
agents_stat_summary_by_year['stay'] = agents_stat_summary_by_year['stay_income_low'] + agents_stat_summary_by_year['stay_income_medium'] +  + agents_stat_summary_by_year['stay_income_high']
agents_stat_summary_by_year['total_pop'] = agents_stat_summary_by_year['New Comers'] + agents_stat_summary_by_year['stay']
agents_stat_summary_by_year['New Comers_income_low_ratio'] = agents_stat_summary_by_year['New Comers_income_low']/agents_stat_summary_by_year['total_pop']
agents_stat_summary_by_year['New Comers_income_medium_ratio'] = agents_stat_summary_by_year['New Comers_income_medium']/agents_stat_summary_by_year['total_pop']
agents_stat_summary_by_year['New Comers_income_high_ratio'] = agents_stat_summary_by_year['New Comers_income_high']/agents_stat_summary_by_year['total_pop']

agents_stat_summary_by_year['stay_income_low_ratio'] = agents_stat_summary_by_year['stay_income_low']/agents_stat_summary_by_year['total_pop']
agents_stat_summary_by_year['stay_income_medium_ratio'] = agents_stat_summary_by_year['stay_income_medium']/agents_stat_summary_by_year['total_pop']
agents_stat_summary_by_year['stay_income_high_ratio'] = agents_stat_summary_by_year['stay_income_high']/agents_stat_summary_by_year['total_pop']


In [10]:
# Generate count status graph
def get_status_graph(bldg_status):
  #"status"
  fig = px.bar(bldg_status, x='status', y="count",width=500,height=200,template='plotly_white',title="Current Building Status",category_orders={'status':['Building before', 'Construction', 'Building after']})
  fig.update_layout(margin={"r":0,"t":35,"l":0,"b":0,"pad":0},
  showlegend=True,

  # font=dict(
  #     size=15,
  # )
  )
  fig.update_yaxes(automargin=True)
  fig.update_yaxes(range=[0, 70])
  return fig


# Dot Matrix

In [11]:

def exportFigure_sq(list_,date_,MatrixPlot):

    # Use column names of df for the different parameters x, y, color, ...
    MatrixPlot['Who'] = list_
    fig = px.scatter(MatrixPlot, x="x", y="y", color="Who",
                     title=f'Staying vs Leaving Snapshot {date_}',
                     labels={"Who":"Legend"}, # customize axis label
                     template='simple_white',color_discrete_map={
                         "Staying": "orange",
                         "New Comers": "blue",
                         "Future Units": "hsv(0,0%,95%)"})
    fig.update_layout(width=550,height=450,margin=dict(l=0, r=0, t=50, b=0),legend=dict(orientation="h", yanchor="top",y=0.99,xanchor="left",x=0.01))
    fig.update_yaxes(showticklabels=False,visible=False)
    fig.update_xaxes(showticklabels=False,visible=False)
    fig.update_traces(marker=dict(size=6))

    return fig

In [12]:

    #fig.write_image(f'matrixPlot_{ticNumber}.png')

#create an empty dataset with agent coutn
num_of_agents = 3481
MatrixPlot = pd.DataFrame({'AgentID':range(0,num_of_agents)})
#Number of Columns choose:
numOfColumns=59
numOfRows = 59
# set the columns to display the column number
MatrixPlot['x'] = list(range(1,numOfColumns+1))*numOfRows
#prepare the y column (row number)
y = [[i]*numOfColumns for i in range(1,numOfRows+1)]
temp = []
for i in range(0,numOfRows):
    temp +=y[i]
y = temp
MatrixPlot['y'] = y

agents_time_new_Comers_stay = agents_stat_summary_by_year[['year','New Comers','stay']].fillna(0)
agents_time_new_Comers_stay['emptyPalces'] = num_of_agents -(agents_time_new_Comers_stay['stay'] +agents_time_new_Comers_stay['New Comers'])

list_ = ['Staying']* agents_time_new_Comers_stay.loc[20,'stay'].astype(int)
list_ = list_ + ['New Comers']*agents_time_new_Comers_stay.loc[20,'New Comers'].astype(int)
list_ = list_ + ['Future Units']*agents_time_new_Comers_stay.loc[20,'emptyPalces'].astype(int)
dotMAtrixFig1 = exportFigure_sq(list_,('temp'),MatrixPlot)



In [14]:
#fig = px.line(agents_stat_summary_by_year, x="year"})

def renters_owners_graph(year_q):
  rent_own_new_stay = agents_stat_summary_by_year[['year','New Comers_rent','New Comers_own','stay_rent','stay_own']].copy().fillna(0)
  end_index = rent_own_new_stay[rent_own_new_stay['year']==year_q].index.values[0]
  selected_indexes = range(0,end_index+1)
  rent_own_new_stayselected = rent_own_new_stay[rent_own_new_stay.index.isin(selected_indexes)]

  fig = go.Figure()

  fig.add_trace(go.Scatter(
      x=year_ranges,
      y=rent_own_new_stayselected['New Comers_rent'],
      legendgroup="New Comers",  # this can be any string, not just "group"
      legendgrouptitle_text="New Comers",
      name="Renters",
      mode="lines",
      line=dict(color='royalblue', width=1)
  ))


  fig.add_trace(go.Scatter(
      x=year_ranges,
      y=rent_own_new_stayselected['New Comers_own'],
      legendgroup="New Comers",  # this can be any string, not just "group"
      legendgrouptitle_text="New Comers",
      name="Owners",
      mode="lines",
      line=dict(color='royalblue', width=3)
  ))


  fig.add_trace(go.Scatter(
      x=year_ranges,
      y=rent_own_new_stayselected['stay_rent'],
      legendgroup="Staying",  # this can be any string, not just "group"
      legendgrouptitle_text="Staying",
      name="Renters",
      mode="lines",
      line=dict(color='firebrick', width=1)
  ))


  fig.add_trace(go.Scatter(
      x=year_ranges,
      y=rent_own_new_stayselected['stay_own'],
      legendgroup="Staying",  # this can be any string, not just "group"
      legendgrouptitle_text="Staying",
      name="Owners",
      mode="lines",
      line=dict(color='firebrick', width=5)
  ))



  fig.update_layout(title="Staying Laving and Owenrship",template='plotly_white',yaxis = {'title' : "Absolute Numbers"},margin={"r":0,"t":35,"l":0,"b":0,"pad":0},
                    width=1100,height=350,legend=dict(orientation="h", yanchor="top",y=0.99,xanchor="left",x=0.01))
  # fig.update_xaxes(
  #     ticktext=list(year_ranges),
  #     tickvals=list(year_ranges)
  # )
  #fig.update_xaxes(range=list(year_ranges))
  fig.update_xaxes(range = [0,len(year_ranges)])
  fig.update_yaxes(range = [0,2100])
  return fig

# set range
renters_owners_fig = renters_owners_graph('2015 Q1')

# Construction

In [60]:



def construction_typo_graph_generator(construction_typo_v,construction_typo_d):

  try:
    v_a = construction_typo_v[construction_typo_v['project_ty']==1]['count'].values[0]
  except:
    v_a = 0
  try:
    v_r = construction_typo_v[construction_typo_v['project_ty']==2]['count'].values[0]
  except:
    v_r = 0
  try:
    v_rr = construction_typo_v[construction_typo_v['project_ty']==3]['count'].values[0]
  except:
    v_rr = 0

  try:
    d_a = construction_typo_d[construction_typo_d['project_ty']==1]['count'].values[0]
  except:
    d_a = 0
  try:
    d_r = construction_typo_d[construction_typo_d['project_ty']==2]['count'].values[0]
  except:
    d_r = 0
  try:
    d_rr = construction_typo_d[construction_typo_d['project_ty']==3]['count'].values[0]
  except:
    d_rr = 0
  fig = go.Figure()

  fig.add_trace(go.Indicator(
      mode = "number+delta",
      value = v_a,
      domain = {'x': [0.05, 0.25], 'y': [0.7, 0.9]},
      number_font_color="#2052a7",
      
      delta = {'reference': d_a, 'relative': True,'valueformat':'.2%'}))

  fig.add_trace(go.Indicator(
      mode = "number+delta",
      value = v_r,
      domain = {'x': [0.3, 0.5], 'y': [0.7, 0.9]},
      number_font_color="#4c84cb",
      delta = {'reference': d_r, 'relative': True,'valueformat':'.2%'}))

  fig.add_trace(go.Indicator(
      mode = "number+delta",
      value = v_rr,
      domain = {'x': [0.6, 0.80], 'y': [0.7, 0.9]},
      number_font_color="#87b1eb",
      delta = {'reference': d_rr, 'relative': True,'valueformat':'.2%'}))


  # Add images
  fig.add_layout_image(
          dict(
              source="https://raw.githubusercontent.com/Shai2u/demographic_estimation_dashboard_article/main/assets/graphics/Building_Typology.jpg",
              xref="paper", yref="paper",
              x=0, y=0,
              sizex=1, sizey=1,
              xanchor="left", yanchor="bottom",
              layer="below")
  )

  fig.update_layout(template="plotly_white",margin={"r":0,"t":0,"l":0,"b":0,"pad":0},width=500,height=200)
  return fig


In [53]:
d = pd.to_datetime('2015-01-01')
d2 = d - datetime.timedelta(days=180)
bldgs1 = simulatedBldgsGdf[(simulatedBldgsGdf['start_date']< d ) & (simulatedBldgsGdf['end_date']> d )].copy().reset_index(drop=True)

bldgs_constr = bldgs1[bldgs1['status']=='Construction'].reset_index()
#Add all columns
if len(bldg_status)<3:
  if len(bldg_status[bldg_status['status'].isin(['Building before'])])==0:
    bldg_status = pd.concat([bldg_status,pd.DataFrame({'status':['Building before'],'count':[0]})]).reset_index(drop=True)
  if len(bldg_status[bldg_status['status'].isin(['Construction'])])==0:
    bldg_status = pd.concat([bldg_status,pd.DataFrame({'status':['Construction'],'count':[0]})]).reset_index(drop=True)
  if len(bldg_status[bldg_status['status'].isin(['Building after'])])==0:
    bldg_status = pd.concat([bldg_status,pd.DataFrame({'status':['Building after'],'count':[0]})]).reset_index(drop=True)

construction_typo_v = bldgs_constr['project_ty'].value_counts().to_frame().reset_index()
construction_typo_v.rename(columns={'index':'project_ty','project_ty':'count'},inplace=True)
bldgs2 = simulatedBldgsGdf[(simulatedBldgsGdf['start_date']< d2 ) & (simulatedBldgsGdf['end_date']> d2 )].copy().reset_index(drop=True)

bldgs_constr = bldgs2[bldgs2['status']=='Construction'].reset_index()

construction_typo_d = bldgs_constr['project_ty'].value_counts().to_frame().reset_index()
construction_typo_d.rename(columns={'index':'project_ty','project_ty':'count'},inplace=True)
construction_typo_graph = construction_typo_graph_generator(construction_typo_v,construction_typo_d)

# Suburst

In [54]:
def demographic_sunburst(year_q,agents_stay_age_income,colorDict):
  title_ = f'{year_q} : Population composition snapshot'
  # color_discrete_map = color_group_map_
  fig = px.sunburst(agents_stay_age_income, path=['Stay or leave','Age group','Income category'], title=title_)

  labels_text = fig.data[0].labels.tolist()
  colorLabels = tuple(colorDict[item] for item in labels_text)
  fig.data[0].marker.colors = colorLabels
  fig.update_traces(textinfo="label+percent entry")
  fig.update_layout(showlegend=False, margin=dict(l=0, r=0, t=50, b=0), width=500, height=500)
  return fig


In [55]:

year_makrs = [year for year in np.arange(2015,2031,0.5)]
years_with_q2_makrs = []
for year in year_makrs:
    d = pd.to_datetime(f'{int(year)}-07-01')
    if year % 1 == 0:
        d = pd.to_datetime(f'{int(year)}-01-01')
    years_with_q2_makrs.append(d)
    
d = pd.to_datetime('2015-01-01')
bldgs  = simulatedBldgsGdf[(simulatedBldgsGdf['start_date']< d ) & (simulatedBldgsGdf['end_date']> d )].copy().reset_index(drop=True)
bldgs1 = bldgs.copy()
agents_synced_buildings_to_date = pd.merge(agents_track_status,bldgs1[['project_nu','status','start_date']],left_on=['ProjNumber','bld_status'], right_on=['project_nu','status'])
agents_synced_buildings_to_date_stay_new = agents_synced_buildings_to_date[agents_synced_buildings_to_date['status_x']!='Leave'].drop_duplicates().reset_index(drop=True)

age_grown1 = d.year-years_with_q2_makrs[0].year
agents_synced_buildings_to_date_stay_new.loc[agents_synced_buildings_to_date_stay_new['status_x']=='stay','age'] = agents_synced_buildings_to_date_stay_new.loc[agents_synced_buildings_to_date_stay_new['status_x']=='stay','age'] + age_grown1
agents_synced_buildings_to_date_stay_new.loc[agents_synced_buildings_to_date_stay_new['status_x']=='New Comers','age'] = agents_synced_buildings_to_date_stay_new.loc[agents_synced_buildings_to_date_stay_new['status_x']=='New Comers'].apply(lambda p: p['age'] + (d.year - p['start_date'].year),axis=1)

agents_synced_buildings_to_date_stay_new['age_group'] = pd.cut(agents_synced_buildings_to_date_stay_new['age'], [0,44,64,84,130],right=True, labels=["18-44", "45-64", "65-84", "85+"],ordered=True)
agents_stay_age_income =agents_synced_buildings_to_date_stay_new[['status_x','age_group','income_cat']].reset_index(drop=True).rename(columns={'status_x':'Stay or leave','age_group':'Age group','income_cat':'Income category'})

In [56]:
population_sunburst_graph_init = demographic_sunburst(2019,agents_stay_age_income,colorDict)

# Bubble Figure

In [57]:
def bubble_age_income_stay_time(StayAgeIncome_ReftedDate,NewComersAgeIncome_RefDate,StayAgeIncome_SelectedDate,NewComersAgeIncome_SelectedDate,ref_date_,date_):
  #Reference plot
  fig = make_subplots(rows=1, cols=2,subplot_titles=(f"Reference Date {ref_date_}", f"Current Date {date_}"))
  fig.add_trace(
      go.Scatter(x=NewComersAgeIncome_RefDate['Income category'], y=NewComersAgeIncome_RefDate['Age group'], mode="markers",name='New Comers',marker=dict(size=NewComersAgeIncome_RefDate['ratio']*250,opacity=0.5,color='blue'),showlegend=True,hovertemplate='Households %{text}',text = list(NewComersAgeIncome_RefDate['units'])),
      row=1, col=1
  )
  fig.add_trace(
      go.Scatter(x=StayAgeIncome_ReftedDate['Income category'], y=StayAgeIncome_ReftedDate['Age group'], mode="markers",name='Staying',marker=dict(size=StayAgeIncome_ReftedDate['ratio']*250,opacity=0.5,color='red'),showlegend=True,hovertemplate='%Households {text}',text = list(StayAgeIncome_ReftedDate['units'])),
      row=1, col=1
  )
  
  fig.add_trace(
      go.Scatter(x=NewComersAgeIncome_SelectedDate['Income category'], y=NewComersAgeIncome_SelectedDate['Age group'], mode="markers",name='New Comers',marker=dict(size=NewComersAgeIncome_SelectedDate['ratio']*250,opacity=0.5,color='blue'),showlegend=True,hovertemplate='Households %{text}',text = list(NewComersAgeIncome_SelectedDate['units'])),
      row=1, col=2
  )
  fig.add_trace(
      go.Scatter(x=StayAgeIncome_SelectedDate['Income category'], y=StayAgeIncome_SelectedDate['Age group'], mode="markers",name='Staying',marker=dict(size=StayAgeIncome_SelectedDate['ratio']*250,opacity=0.5,color='red'),showlegend=True,hovertemplate='%Households {text}',text = list(StayAgeIncome_SelectedDate['units'])),
      row=1, col=2
  )
  # text = list(StayAgeIncome_SelectedDate['units'])
  # hover_text = '%{text}'


  #hovertemplate="name: %{y}%{x}<br>number: %{marker.symbol}<extra></extra>")


  fig.update_layout(height=475, width=550, title_text="Side By Side Subplots", margin=dict(l=0, r=0, t=50, b=0),legend=dict(orientation="h", yanchor="top",y=0.99,xanchor="left",x=0.01),template='plotly_white')
  return fig

# get date from function
d = pd.to_datetime('2015-01-01')
bldgs  = simulatedBldgsGdf[(simulatedBldgsGdf['start_date']< d ) & (simulatedBldgsGdf['end_date']> d )].copy().reset_index(drop=True)
bldgs1 = bldgs.copy()

agents_synced_buildings_to_date = pd.merge(agents_track_status,bldgs1[['project_nu','status','start_date']],left_on=['ProjNumber','bld_status'], right_on=['project_nu','status'])
agents_synced_buildings_to_date_stay_new = agents_synced_buildings_to_date[agents_synced_buildings_to_date['status_x']!='Leave'].drop_duplicates().reset_index(drop=True)

#age the agents
age_grown1 = d.year-years_with_q2_makrs[0].year
agents_synced_buildings_to_date_stay_new.loc[agents_synced_buildings_to_date_stay_new['status_x']=='stay','age'] = agents_synced_buildings_to_date_stay_new.loc[agents_synced_buildings_to_date_stay_new['status_x']=='stay','age'] + age_grown1
agents_synced_buildings_to_date_stay_new.loc[agents_synced_buildings_to_date_stay_new['status_x']=='New Comers','age'] = agents_synced_buildings_to_date_stay_new.loc[agents_synced_buildings_to_date_stay_new['status_x']=='New Comers'].apply(lambda p: p['age'] + (d.year - p['start_date'].year),axis=1)

#set categories to agents ages
agents_synced_buildings_to_date_stay_new['age_group'] = pd.cut(agents_synced_buildings_to_date_stay_new['age'], [0,44,64,84,130],right=True, labels=["18-44", "45-64", "65-84", "85+"],ordered=True)
agents_stay_age_income =agents_synced_buildings_to_date_stay_new[['status_x','age_group','income_cat']].reset_index(drop=True).rename(columns={'status_x':'Stay or leave','age_group':'Age group','income_cat':'Income category'})
#population_sunburst_graph = demographic_sunburst(2019,agents_stay_age_income,colorDict)
agents_stay_age_income['units'] = 1
agents_stay_age_income_count = agents_stay_age_income.groupby(['Stay or leave','Age group','Income category']).agg({'units':'count'}).reset_index()
new_comers_age_income_count = agents_stay_age_income_count[agents_stay_age_income_count['Stay or leave']=='New Comers'].reset_index(drop=True)
stay_age_income_count = agents_stay_age_income_count[agents_stay_age_income_count['Stay or leave']=='stay'].reset_index(drop=True)
total = new_comers_age_income_count['units'].sum()+ stay_age_income_count['units'].sum()
new_comers_age_income_count['ratio'] = new_comers_age_income_count['units']/total
stay_age_income_count['ratio'] = stay_age_income_count['units']/total

d_ref = pd.to_datetime('2015-01-01')
bldgs  = simulatedBldgsGdf[(simulatedBldgsGdf['start_date']< d_ref ) & (simulatedBldgsGdf['end_date']> d_ref )].copy().reset_index(drop=True)
bldgs1 = bldgs.copy()

agents_synced_buildings_ref_date = pd.merge(agents_track_status,bldgs1[['project_nu','status','start_date']],left_on=['ProjNumber','bld_status'], right_on=['project_nu','status'])
agents_synced_buildings_ref_date_stay_new = agents_synced_buildings_ref_date[agents_synced_buildings_ref_date['status_x']!='Leave'].drop_duplicates().reset_index(drop=True)

#age the agents
age_grown1 = d_ref.year-years_with_q2_makrs[0].year
agents_synced_buildings_ref_date_stay_new.loc[agents_synced_buildings_ref_date_stay_new['status_x']=='stay','age'] = agents_synced_buildings_ref_date_stay_new.loc[agents_synced_buildings_ref_date_stay_new['status_x']=='stay','age'] + age_grown1
agents_synced_buildings_ref_date_stay_new.loc[agents_synced_buildings_ref_date_stay_new['status_x']=='New Comers','age'] = agents_synced_buildings_ref_date_stay_new.loc[agents_synced_buildings_ref_date_stay_new['status_x']=='New Comers'].apply(lambda p: p['age'] + (d_ref.year - p['start_date'].year),axis=1)

#set categories to agents ages
agents_synced_buildings_ref_date_stay_new['age_group'] = pd.cut(agents_synced_buildings_ref_date_stay_new['age'], [0,44,64,84,130],right=True, labels=["18-44", "45-64", "65-84", "85+"],ordered=True)
agents_stay_age_income_ref =agents_synced_buildings_ref_date_stay_new[['status_x','age_group','income_cat']].reset_index(drop=True).rename(columns={'status_x':'Stay or leave','age_group':'Age group','income_cat':'Income category'})
#population_sunburst_graph = demographic_sunburst(2019,agents_stay_age_income,colorDict)
agents_stay_age_income_ref['units'] = 1
agents_stay_age_income_ref = agents_stay_age_income_ref.groupby(['Stay or leave','Age group','Income category']).agg({'units':'count'}).reset_index()
new_comers_age_income_count_ref = agents_stay_age_income_ref[agents_stay_age_income_ref['Stay or leave']=='New Comers'].reset_index(drop=True)
stay_age_income_count_ref = agents_stay_age_income_ref[agents_stay_age_income_ref['Stay or leave']=='stay'].reset_index(drop=True)
total_ref = new_comers_age_income_count_ref['units'].sum()+ stay_age_income_count_ref['units'].sum()
new_comers_age_income_count_ref['ratio'] = new_comers_age_income_count_ref['units']/total_ref
stay_age_income_count_ref['ratio'] = stay_age_income_count_ref['units']/total_ref
bubble_age_income = bubble_age_income_stay_time(stay_age_income_count_ref,new_comers_age_income_count_ref,stay_age_income_count,new_comers_age_income_count,'2015 Q1','2015 Q1')


In [58]:
agents_stay_age_income_ref

,Stay or leave,Age group,Income category,units
0,stay,18-44,Low,266
1,stay,18-44,Medium,225
2,stay,18-44,High,0
3,stay,45-64,Low,89
4,stay,45-64,Medium,85
5,stay,45-64,High,0
6,stay,65-84,Low,75
7,stay,65-84,Medium,85
8,stay,65-84,High,0
9,stay,85+,Low,40


# Dash

In [59]:

def prepare_dot_matrix(q_date_for_dot_matrix):
    # #agents_stat_summary_by_year = pd.read_csv('https://raw.githubusercontent.com/Shai2u/demographic_estimation_dashboard_article/main/dashboard/data/yearly_stats_for_dashboard.csv')

    # agents_stat_summary_by_year['New Comers'] = agents_stat_summary_by_year['New Comers_income_low'] + agents_stat_summary_by_year['New Comers_income_medium'] +  + agents_stat_summary_by_year['New Comers_income_high']
    # agents_stat_summary_by_year['stay'] = agents_stat_summary_by_year['stay_income_low'] + agents_stat_summary_by_year['stay_income_medium'] +  + agents_stat_summary_by_year['stay_income_high']
    # agents_stat_summary_by_year['total_pop'] = agents_stat_summary_by_year['New Comers'] + agents_stat_summary_by_year['stay']
    # agents_stat_summary_by_year['New Comers_income_low_ratio'] = agents_stat_summary_by_year['New Comers_income_low']/agents_stat_summary_by_year['total_pop']
    # agents_stat_summary_by_year['New Comers_income_medium_ratio'] = agents_stat_summary_by_year['New Comers_income_medium']/agents_stat_summary_by_year['total_pop']
    # agents_stat_summary_by_year['New Comers_income_high_ratio'] = agents_stat_summary_by_year['New Comers_income_high']/agents_stat_summary_by_year['total_pop']

    # agents_stat_summary_by_year['stay_income_low_ratio'] = agents_stat_summary_by_year['stay_income_low']/agents_stat_summary_by_year['total_pop']
    # agents_stat_summary_by_year['stay_income_medium_ratio'] = agents_stat_summary_by_year['stay_income_medium']/agents_stat_summary_by_year['total_pop']
    # agents_stat_summary_by_year['stay_income_high_ratio'] = agents_stat_summary_by_year['stay_income_high']/agents_stat_summary_by_year['total_pop']

    #create an empty dataset with agent coutn
    num_of_agents = 3481
    MatrixPlot = pd.DataFrame({'AgentID':range(0,num_of_agents)})
    #Number of Columns choose:
    numOfColumns=59
    numOfRows = 59
    # set the columns to display the column number
    MatrixPlot['x'] = list(range(1,numOfColumns+1))*numOfRows
    #prepare the y column (row number)
    y = [[i]*numOfColumns for i in range(1,numOfRows+1)]
    temp = []
    for i in range(0,numOfRows):
        temp +=y[i]
    y = temp
    MatrixPlot['y'] = y
    agents_stat_short = agents_stat_summary_by_year[['year','New Comers','stay']].copy().fillna(0)
    agents_stat_short['emptyPalces'] =agents_stat_short.apply(lambda p:num_of_agents-(p['stay']+p['New Comers']), axis=1)
    stay_count = int(agents_stat_short.loc[agents_stat_short['year']==q_date_for_dot_matrix,'stay'].values[0])
    new_comers = int(agents_stat_short.loc[agents_stat_short['year']==q_date_for_dot_matrix,'New Comers'].values[0])
    empty_space = int(agents_stat_short.loc[agents_stat_short['year']==q_date_for_dot_matrix,'emptyPalces'].values[0])
    list_ = ['Staying']* stay_count
    list_ = list_ + ['New Comers'] * new_comers
    list_ = list_ + ['Future Units'] * empty_space
    dotMAtrixFig = exportFigure_sq(list_,(q_date_for_dot_matrix),MatrixPlot)
    return dotMAtrixFig
    #dotMAtrixFig = dotMAtrixFig1

line_style = dict(weight=2, opacity=1, color='blue', fillOpacity=0,dashArray="10 10")

classes = ['Building before', 'Construction', 'Building after']
colorscale = ['#FFEDA0', '#FEB24C', '#FC4E2A']
style = dict(weight=2, opacity=1, color='white', fillOpacity=0.7)
colorbar = dlx.categorical_colorbar(categories=classes, colorscale=colorscale, width=300, height=30, position="bottomleft")

style_handle = assign("""function(feature, context){
    const {classes, colorscale, style, colorProp} = context.props.hideout;  // get props from hideout
    const value = feature.properties[colorProp];  // get value the determines the color
    for (let i = 0; i < classes.length; ++i) {
        if (value == classes[i]) {
            style.fillColor = colorscale[i];  // set the fill color according to the class
        }
    }
    return style;
}""")


mapObj = dl.Map([dl.TileLayer(url=cartoUrl, maxZoom=20, attribution=attribution),dl.GeoJSON(data=statisticalStatsJson, options={'style':line_style}),dl.GeoJSON(data = simulatedBldgsJson,options = {'style':style_handle}, id='simulatedBldgs',hideout=dict(colorscale=colorscale, classes=classes, style=style, colorProp="status")),colorbar],center=[32.0272,34.7444], zoom=16, style={'width': '100%', 'height': '600px'})


d = pd.to_datetime('2015-01-01')
bldgs1 = simulatedBldgsGdf[(simulatedBldgsGdf['start_date']< d ) & (simulatedBldgsGdf['end_date']> d )].copy().reset_index(drop=True)
bldg_status = bldgs1['status'].value_counts().to_frame().reset_index()
bldg_status.rename(columns={'index':'status','status':'count'},inplace=True)
bldg_status_count = get_status_graph(bldg_status)


#create an empty dataset with agent coutn
num_of_agents = 3481
MatrixPlot = pd.DataFrame({'AgentID':range(0,num_of_agents)})
#Number of Columns choose:
numOfColumns=59
numOfRows = 59
# set the columns to display the column number
MatrixPlot['x'] = list(range(1,numOfColumns+1))*numOfRows
#prepare the y column (row number)
y = [[i]*numOfColumns for i in range(1,numOfRows+1)]
temp = []
for i in range(0,numOfRows):
    temp +=y[i]
y = temp
MatrixPlot['y'] = y

agents_time_new_Comers_stay = agents_stat_summary_by_year[['year','New Comers','stay']].fillna(0)
agents_time_new_Comers_stay['emptyPalces'] = num_of_agents -(agents_time_new_Comers_stay['stay'] +agents_time_new_Comers_stay['New Comers'])


dashboard_page =  html.Div([
  html.Div([
    html.Div([
              #To Do Add static legend, that won't chnaged based on years and statistical stats
              html.Div(['Place2',mapObj
              
              ])
            ],style={'width': '34%', 'display': 'inline-block'}),

        html.Div([
            html.Table([
                html.Tr([html.Td(['Place1'],id='selectedDate',style={'direction':'rtl','width':'10%'}),
                    html.Td([dcc.RangeSlider(id='years-slider',
             min=2015,
             max=2030,
             value=[2015,2015],
             marks={str(year):str(year) for year in np.arange(2015,2030,1)},
                                                    step=0.5
                                                    )
                    ],style={'direction':'rtl','width':'90%'})
                    
                ])
            ],style={'direction':'rtl','width':'100%'}),html.Div([dcc.Graph(id='time_seiries_graph',figure=renters_owners_fig)]),
            html.Div([html.Div([dcc.Graph(id='typo_count', figure=construction_typo_graph)],style={'width': '49%', 'display': 'inline-block'}),
                      html.Div([dcc.Graph(id='status_count',figure=bldg_status_count)],style={'width': '49%', 'display': 'inline-block'})
                      
                      ])




                                                    ], style={'width': '66%', 'float': 'right', 'display': 'inline-block'}),
              html.Div([
                            html.Div([
                                #To Do Add static legend, that won't chnaged based on years and statistical stats
                                html.Div([
                                          dcc.Graph(id='bubble_graph',figure=bubble_age_income)])],style={'width': '34%', 'display': 'inline-block'}),
                      html.Div([
                                #To Do Add static legend, that won't chnaged based on years and statistical stats
                                html.Div([dcc.Graph(id='population_sunburst_fig',figure= population_sunburst_graph_init)])],style={'width': '33%', 'display': 'inline-block'}),
                      html.Div([
                                #To Do Add static legend, that won't chnaged based on years and statistical stats
                                html.Div([dcc.Graph(id='dot_matrix_fig',figure=dotMAtrixFig1),html.Div(id='test',children='test')])],style={'width': '33%', 'display': 'inline-block'}),


            ])
  ], style={
        'borderBottom': 'thin lightgrey solid',
        'backgroundColor': 'rgb(250, 250, 250)',
        'padding': '10px 5px'})
        #dcc.Store(id='eng_heb_data')
])
app = JupyterDash(__name__,suppress_callback_exceptions=True,prevent_initial_callbacks=True)
app.layout = html.Div([

    dashboard_page
])

@app.callback(
    Output('selectedDate', 'children'),
    Output('simulatedBldgs', 'data'),
    Output('status_count','figure'),
    Output('dot_matrix_fig','figure'),
    Output('time_seiries_graph','figure'),
    Output('typo_count','figure'),
    Output('population_sunburst_fig','figure'),
    Output('bubble_graph','figure'),
    Input('years-slider', 'value')
    # Output('test','children'),
    #Output('dot_matrix_fig','figure'),

)
def update_output_div(input_value):
    #simulatedBldgsGdf
    ref_year = int(input_value[0])
    selected_year = int(input_value[1])
    date_return = f'Q3 {selected_year}'
    q_date_for_dot_matrix = f'{selected_year} Q3'
    d = pd.to_datetime(f'{selected_year}-07-01')
    if input_value[1] % 1 == 0:
        d = pd.to_datetime(f'{selected_year}-01-01')
        date_return = f'Q1 {selected_year}'
        q_date_for_dot_matrix = f'{selected_year} Q1'
    #reference date
    d_ref = pd.to_datetime(f'{ref_year}-07-01')
    ref_q_date = f'Q3 {ref_year}'
    if input_value[0] % 1 == 0:
          d_ref = pd.to_datetime(f'{ref_year}-01-01')
          ref_q_date = f'Q1 {ref_year}'



    bldgs  = simulatedBldgsGdf[(simulatedBldgsGdf['start_date']< d ) & (simulatedBldgsGdf['end_date']> d )].copy().reset_index(drop=True)
    bldgs1 = bldgs.copy()
    bldgs['start_date'] = bldgs['start_date'].astype(str)
    bldgs['end_date'] = bldgs['end_date'].astype(str)
    bldgsJson = json.loads(bldgs.to_json())


    #construction
    bldg_status = bldgs1['status'].value_counts().to_frame().reset_index()
    bldg_status.rename(columns={'index':'status','status':'count'},inplace=True)
    if len(bldg_status)<3:
      if len(bldg_status[bldg_status['status'].isin(['Building before'])])==0:
        bldg_status = pd.concat([bldg_status,pd.DataFrame({'status':['Building before'],'count':[0]})]).reset_index(drop=True)
      if len(bldg_status[bldg_status['status'].isin(['Construction'])])==0:
        bldg_status = pd.concat([bldg_status,pd.DataFrame({'status':['Construction'],'count':[0]})]).reset_index(drop=True)
      if len(bldg_status[bldg_status['status'].isin(['Building after'])])==0:
        bldg_status = pd.concat([bldg_status,pd.DataFrame({'status':['Building after'],'count':[0]})]).reset_index(drop=True)
    bldg_status_count = get_status_graph(bldg_status)
    
    d2 = d - datetime.timedelta(days=180)
    bldgs2 = simulatedBldgsGdf[(simulatedBldgsGdf['start_date']< d ) & (simulatedBldgsGdf['end_date']> d )].copy().reset_index(drop=True)

    bldgs_constr = bldgs2[bldgs2['status']=='Construction'].reset_index()

    construction_typo_v = bldgs_constr['project_ty'].value_counts().to_frame().reset_index()
    construction_typo_v.rename(columns={'index':'project_ty','project_ty':'count'},inplace=True)

    bldgs3 = simulatedBldgsGdf[(simulatedBldgsGdf['start_date']< d2 ) & (simulatedBldgsGdf['end_date']> d2 )].copy().reset_index(drop=True)
    bldgs_constr = bldgs3[bldgs3['status']=='Construction'].reset_index()

    construction_typo_d = bldgs_constr['project_ty'].value_counts().to_frame().reset_index()
    construction_typo_d.rename(columns={'index':'project_ty','project_ty':'count'},inplace=True)
    construction_typo_graph = construction_typo_graph_generator(construction_typo_v,construction_typo_d)
    #bldg_status_count = get_status_graph(bldg_status)


    # Dot Matrix

    dotMAtrixFig1 = prepare_dot_matrix(q_date_for_dot_matrix)
    #renters Owners
    renters_owners_fig = renters_owners_graph(q_date_for_dot_matrix)


    #sunburst

    bldgs  = simulatedBldgsGdf[(simulatedBldgsGdf['start_date']< d ) & (simulatedBldgsGdf['end_date']> d )].copy().reset_index(drop=True)
    bldgs1 = bldgs.copy()
    agents_synced_buildings_to_date = pd.merge(agents_track_status,bldgs1[['project_nu','status','start_date']],left_on=['ProjNumber','bld_status'], right_on=['project_nu','status'])
    agents_synced_buildings_to_date_stay_new = agents_synced_buildings_to_date[agents_synced_buildings_to_date['status_x']!='Leave'].drop_duplicates().reset_index(drop=True)

    age_grown1 = d.year-years_with_q2_makrs[0].year
    agents_synced_buildings_to_date_stay_new.loc[agents_synced_buildings_to_date_stay_new['status_x']=='stay','age'] = agents_synced_buildings_to_date_stay_new.loc[agents_synced_buildings_to_date_stay_new['status_x']=='stay','age'] + age_grown1
    agents_synced_buildings_to_date_stay_new.loc[agents_synced_buildings_to_date_stay_new['status_x']=='New Comers','age'] = agents_synced_buildings_to_date_stay_new.loc[agents_synced_buildings_to_date_stay_new['status_x']=='New Comers'].apply(lambda p: p['age'] + (d.year - p['start_date'].year),axis=1)

    agents_synced_buildings_to_date_stay_new['age_group'] = pd.cut(agents_synced_buildings_to_date_stay_new['age'], [0,44,64,84,130],right=True, labels=["18-44", "45-64", "65-84", "85+"],ordered=True)
    agents_stay_age_income =agents_synced_buildings_to_date_stay_new[['status_x','age_group','income_cat']].reset_index(drop=True).rename(columns={'status_x':'Stay or leave','age_group':'Age group','income_cat':'Income category'})
    population_sunburst_graph = demographic_sunburst(date_return,agents_stay_age_income,colorDict)


    #bubble graphs Selected
    bldgs  = simulatedBldgsGdf[(simulatedBldgsGdf['start_date']< d ) & (simulatedBldgsGdf['end_date']> d )].copy().reset_index(drop=True)
    bldgs1 = bldgs.copy()

    agents_synced_buildings_to_date = pd.merge(agents_track_status,bldgs1[['project_nu','status','start_date']],left_on=['ProjNumber','bld_status'], right_on=['project_nu','status'])
    agents_synced_buildings_to_date_stay_new = agents_synced_buildings_to_date[agents_synced_buildings_to_date['status_x']!='Leave'].drop_duplicates().reset_index(drop=True)

    #age the agents
    age_grown1 = d.year-years_with_q2_makrs[0].year
    agents_synced_buildings_to_date_stay_new.loc[agents_synced_buildings_to_date_stay_new['status_x']=='stay','age'] = agents_synced_buildings_to_date_stay_new.loc[agents_synced_buildings_to_date_stay_new['status_x']=='stay','age'] + age_grown1
    agents_synced_buildings_to_date_stay_new.loc[agents_synced_buildings_to_date_stay_new['status_x']=='New Comers','age'] = agents_synced_buildings_to_date_stay_new.loc[agents_synced_buildings_to_date_stay_new['status_x']=='New Comers'].apply(lambda p: p['age'] + (d.year - p['start_date'].year),axis=1)

    #set categories to agents ages
    agents_synced_buildings_to_date_stay_new['age_group'] = pd.cut(agents_synced_buildings_to_date_stay_new['age'], [0,44,64,84,130],right=True, labels=["18-44", "45-64", "65-84", "85+"],ordered=True)
    agents_stay_age_income =agents_synced_buildings_to_date_stay_new[['status_x','age_group','income_cat']].reset_index(drop=True).rename(columns={'status_x':'Stay or leave','age_group':'Age group','income_cat':'Income category'})
    #population_sunburst_graph = demographic_sunburst(2019,agents_stay_age_income,colorDict)
    agents_stay_age_income['units'] = 1
    agents_stay_age_income_count = agents_stay_age_income.groupby(['Stay or leave','Age group','Income category']).agg({'units':'count'}).reset_index()
    new_comers_age_income_count = agents_stay_age_income_count[agents_stay_age_income_count['Stay or leave']=='New Comers'].reset_index(drop=True)
    stay_age_income_count = agents_stay_age_income_count[agents_stay_age_income_count['Stay or leave']=='stay'].reset_index(drop=True)
    total = new_comers_age_income_count['units'].sum()+ stay_age_income_count['units'].sum()
    new_comers_age_income_count['ratio'] = new_comers_age_income_count['units']/total
    stay_age_income_count['ratio'] = stay_age_income_count['units']/total

    #reference selection
    bldgs  = simulatedBldgsGdf[(simulatedBldgsGdf['start_date']< d_ref ) & (simulatedBldgsGdf['end_date']> d_ref )].copy().reset_index(drop=True)
    bldgs1 = bldgs.copy()

    agents_synced_buildings_ref_date = pd.merge(agents_track_status,bldgs1[['project_nu','status','start_date']],left_on=['ProjNumber','bld_status'], right_on=['project_nu','status'])
    agents_synced_buildings_ref_date_stay_new = agents_synced_buildings_ref_date[agents_synced_buildings_ref_date['status_x']!='Leave'].drop_duplicates().reset_index(drop=True)

    #age the agents
    age_grown1 = d_ref.year-years_with_q2_makrs[0].year
    agents_synced_buildings_ref_date_stay_new.loc[agents_synced_buildings_ref_date_stay_new['status_x']=='stay','age'] = agents_synced_buildings_ref_date_stay_new.loc[agents_synced_buildings_ref_date_stay_new['status_x']=='stay','age'] + age_grown1
    agents_synced_buildings_ref_date_stay_new.loc[agents_synced_buildings_ref_date_stay_new['status_x']=='New Comers','age'] = agents_synced_buildings_ref_date_stay_new.loc[agents_synced_buildings_ref_date_stay_new['status_x']=='New Comers'].apply(lambda p: p['age'] + (d_ref.year - p['start_date'].year),axis=1)

    #set categories to agents ages
    agents_synced_buildings_ref_date_stay_new['age_group'] = pd.cut(agents_synced_buildings_ref_date_stay_new['age'], [0,44,64,84,130],right=True, labels=["18-44", "45-64", "65-84", "85+"],ordered=True)
    agents_stay_age_income_ref =agents_synced_buildings_ref_date_stay_new[['status_x','age_group','income_cat']].reset_index(drop=True).rename(columns={'status_x':'Stay or leave','age_group':'Age group','income_cat':'Income category'})
    #population_sunburst_graph = demographic_sunburst(2019,agents_stay_age_income,colorDict)
    agents_stay_age_income_ref['units'] = 1
    agents_stay_age_income_ref = agents_stay_age_income_ref.groupby(['Stay or leave','Age group','Income category']).agg({'units':'count'}).reset_index()
    new_comers_age_income_count_ref = agents_stay_age_income_ref[agents_stay_age_income_ref['Stay or leave']=='New Comers'].reset_index(drop=True)
    stay_age_income_count_ref = agents_stay_age_income_ref[agents_stay_age_income_ref['Stay or leave']=='stay'].reset_index(drop=True)
    total_ref = new_comers_age_income_count_ref['units'].sum()+ stay_age_income_count_ref['units'].sum()
    new_comers_age_income_count_ref['ratio'] = new_comers_age_income_count_ref['units']/total_ref
    stay_age_income_count_ref['ratio'] = stay_age_income_count_ref['units']/total_ref
    bubble_age_income = bubble_age_income_stay_time(stay_age_income_count_ref,new_comers_age_income_count_ref,stay_age_income_count,new_comers_age_income_count,ref_q_date,date_return)



    return date_return,bldgsJson,bldg_status_count,dotMAtrixFig1,renters_owners_fig,construction_typo_graph,population_sunburst_graph,bubble_age_income

## To dos
## Add Tama 38 Construction Current Coutner
app.run_server(mode='external',debug=False,port=8050)

 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [02/Apr/2022 14:32:11] "GET /_alive_0cbf7f4d-dc13-428a-9045-c8983d27712f HTTP/1.1" 200 -


Dash app running on:


<IPython.core.display.Javascript object>

# After Dash

In [ ]:
agents_stat_summary_by_year.head()

,Unnamed: 0,year,New Comers_apartment_size_q1,stay_apartment_size_q1,New Comers_apartment_size_q2,stay_apartment_size_q2,New Comers_apartment_size_q3,stay_apartment_size_q3,New Comers_age_q1,stay_age_q1,New Comers_age_q2,stay_age_q2,New Comers_age_q3,stay_age_q3,New Comers_income_q1,stay_income_q1,New Comers_income_q2,stay_income_q2,New Comers_income_q3,stay_income_q3,New Comers_rent,stay_rent,New Comers_own,stay_own,New Comers_income_low,stay_income_low,New Comers_income_medium,stay_income_medium,New Comers_income_high,stay_income_high,New Comers,stay,total_pop,New Comers_income_low_ratio,New Comers_income_medium_ratio,New Comers_income_high_ratio,stay_income_low_ratio,stay_income_medium_ratio,stay_income_high_ratio
0,0,2015 Q1,NaN,66.0,NaN,73.0,NaN,83.0,NaN,30.0,NaN,42.0,NaN,65.00,NaN,6403.0,NaN,8771.0,NaN,11667.00,NaN,318,NaN,579,NaN,470.0,NaN,427.0,NaN,0.0,NaN,897.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,2015 Q3,NaN,68.0,NaN,73.0,NaN,83.0,NaN,30.0,NaN,42.0,NaN,65.00,NaN,6413.5,NaN,8804.0,NaN,11669.00,NaN,310,NaN,565,NaN,454.0,NaN,421.0,NaN,0.0,NaN,875.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,2016 Q1,NaN,68.0,NaN,73.0,NaN,83.0,NaN,31.0,NaN,43.0,NaN,66.00,NaN,6413.5,NaN,8804.0,NaN,11669.00,NaN,310,NaN,565,NaN,454.0,NaN,421.0,NaN,0.0,NaN,875.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,2016 Q3,NaN,66.0,NaN,73.0,NaN,83.0,NaN,31.0,NaN,43.0,NaN,65.25,NaN,6427.0,NaN,8833.5,NaN,11729.75,NaN,294,NaN,542,NaN,432.0,NaN,404.0,NaN,0.0,NaN,836.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,2017 Q1,82.0,66.0,82.0,73.0,82.0,83.0,32.0,32.0,47.0,44.0,60.0,66.75,18838.105263,6496.0,21097.105263,8934.5,22033.947368,11806.25,11.0,282,14.0,520,0.0,405.0,11.0,397.0,14.0,0.0,25.0,802.0,827.0,0.0,0.013301,0.016929,0.489722,0.480048,0.0


In [ ]:
#fig = px.line(agents_stat_summary_by_year, x="year"})
import plotly.graph_objects as go

fig = go.Figure()



fig.add_trace(go.Scatter(
    x=year_ranges,
    y=agents_stat_summary_by_year['New Comers_apartment_size_q1'],
    legendgroup="New Comers",  # this can be any string, not just "group"
    legendgrouptitle_text="New Comers",
    name="Apartment Size Q1",
    mode="lines",
    line=dict(color='royalblue', width=1)
))


fig.add_trace(go.Scatter(
    x=year_ranges,
    y=agents_stat_summary_by_year['New Comers_apartment_size_q2'],
    legendgroup="New Comers",  # this can be any string, not just "group"
    legendgrouptitle_text="New Comers",
    name="Apartment Size Q2",
    mode="lines",
    line=dict(color='royalblue', width=3)
))


fig.add_trace(go.Scatter(
    x=year_ranges,
    y=agents_stat_summary_by_year['New Comers_apartment_size_q3'],
    legendgroup="New Comers",  # this can be any string, not just "group"
    legendgrouptitle_text="New Comers",
    name="Apartment Size Q3",
    mode="lines",
    line=dict(color='royalblue', width=4)
    
))

fig.add_trace(go.Scatter(
    x=year_ranges,
    y=agents_stat_summary_by_year['stay_apartment_size_q1'],
    legendgroup="Staying",  # this can be any string, not just "group"
    legendgrouptitle_text="Staying",
    name="Apartment Size Q1",
    mode="lines",
    line=dict(color='firebrick', width=1)
))


fig.add_trace(go.Scatter(
    x=year_ranges,
    y=agents_stat_summary_by_year['stay_apartment_size_q2'],
    legendgroup="Staying",  # this can be any string, not just "group"
    legendgrouptitle_text="Staying",
    name="Apartment Size Q2",
    mode="lines",
    line=dict(color='firebrick', width=3)
))


fig.add_trace(go.Scatter(
    x=year_ranges,
    y=agents_stat_summary_by_year['stay_apartment_size_q3'],
    legendgroup="Staying",  # this can be any string, not just "group"
    legendgrouptitle_text="Staying",
    name="Apartment Size Q3",
    mode="lines",
    line=dict(color='firebrick', width=4)
    
))

fig.update_layout(title="Apartment Size  (q1-q3) vs Staying or New Comers",template='plotly_white',yaxis = {'title' : "m<sup>2</sup>"})

fig.show()

# set range

In [ ]:
#fig = px.line(agents_stat_summary_by_year, x="year"})
import plotly.graph_objects as go

fig = go.Figure()



fig.add_trace(go.Scatter(
    x=year_ranges,
    y=agents_stat_summary_by_year['New Comers_age_q1'],
    legendgroup="New Comers",  # this can be any string, not just "group"
    legendgrouptitle_text="New Comers",
    name="Age Q1",
    mode="lines",
    line=dict(color='royalblue', width=1)
))


fig.add_trace(go.Scatter(
    x=year_ranges,
    y=agents_stat_summary_by_year['New Comers_age_q2'],
    legendgroup="New Comers",  # this can be any string, not just "group"
    legendgrouptitle_text="New Comers",
    name="Age Q2",
    mode="lines",
    line=dict(color='royalblue', width=3)
))


fig.add_trace(go.Scatter(
    x=year_ranges,
    y=agents_stat_summary_by_year['New Comers_age_q3'],
    legendgroup="New Comers",  # this can be any string, not just "group"
    legendgrouptitle_text="New Comers",
    name="Age Q3",
    mode="lines",
    line=dict(color='royalblue', width=4)
    
))

fig.add_trace(go.Scatter(
    x=year_ranges,
    y=agents_stat_summary_by_year['stay_age_q1'],
    legendgroup="Staying",  # this can be any string, not just "group"
    legendgrouptitle_text="Staying",
    name="Age Q1",
    mode="lines",
    line=dict(color='firebrick', width=1)
))


fig.add_trace(go.Scatter(
    x=year_ranges,
    y=agents_stat_summary_by_year['stay_age_q2'],
    legendgroup="Staying",  # this can be any string, not just "group"
    legendgrouptitle_text="Staying",
    name="Age Q2",
    mode="lines",
    line=dict(color='firebrick', width=3)
))


fig.add_trace(go.Scatter(
    x=year_ranges,
    y=agents_stat_summary_by_year['stay_age_q3'],
    legendgroup="Staying",  # this can be any string, not just "group"
    legendgrouptitle_text="Staying",
    name="Age Q3",
    mode="lines",
    line=dict(color='firebrick', width=4)
    
))

fig.update_layout(title="age (q1,q2,q3) of olderst person at household",template='plotly_white',yaxis = {'title' : "Average Age"})

fig.show()

# set range

In [ ]:
#fig = px.line(agents_stat_summary_by_year, x="year"})
import plotly.graph_objects as go

fig = go.Figure()



fig.add_trace(go.Scatter(
    x=year_ranges,
    y=agents_stat_summary_by_year['New Comers_income_q1'],
    legendgroup="New Comers",  # this can be any string, not just "group"
    legendgrouptitle_text="New Comers",
    name="Income Q1",
    mode="lines",
    line=dict(color='royalblue', width=1)
))


fig.add_trace(go.Scatter(
    x=year_ranges,
    y=agents_stat_summary_by_year['New Comers_income_q2'],
    legendgroup="New Comers",  # this can be any string, not just "group"
    legendgrouptitle_text="New Comers",
    name="Income Q2",
    mode="lines",
    line=dict(color='royalblue', width=3)
))


fig.add_trace(go.Scatter(
    x=year_ranges,
    y=agents_stat_summary_by_year['New Comers_income_q3'],
    legendgroup="New Comers",  # this can be any string, not just "group"
    legendgrouptitle_text="New Comers",
    name="Income Q3",
    mode="lines",
    line=dict(color='royalblue', width=4)
    
))

fig.add_trace(go.Scatter(
    x=year_ranges,
    y=agents_stat_summary_by_year['stay_income_q1'],
    legendgroup="Staying",  # this can be any string, not just "group"
    legendgrouptitle_text="Staying",
    name="Icome Q1",
    mode="lines",
    line=dict(color='firebrick', width=1)
))


fig.add_trace(go.Scatter(
    x=year_ranges,
    y=agents_stat_summary_by_year['stay_income_q2'],
    legendgroup="Staying",  # this can be any string, not just "group"
    legendgrouptitle_text="Staying",
    name="Income Q2",
    mode="lines",
    line=dict(color='firebrick', width=3)
))


fig.add_trace(go.Scatter(
    x=year_ranges,
    y=agents_stat_summary_by_year['stay_income_q3'],
    legendgroup="Staying",  # this can be any string, not just "group"
    legendgrouptitle_text="Staying",
    name="Income Q3",
    mode="lines",
    line=dict(color='firebrick', width=4)
    
))

fig.update_layout(title="Income (q1-q3) of olderst person at household",template='plotly_white',yaxis = {'title' : "Inocme (New Israeli Shekels)"})

fig.show()

# set range

In [ ]:
rent_own_new_stay = agents_stat_summary_by_year[['year','New Comers_rent','New Comers_own','stay_rent','stay_own']].copy().fillna(0)
end_index = rent_own_new_stay[rent_own_new_stay['year']=='2019 Q1'].index.values[0]
selected_indexes = range(0,end_index+1)
rent_own_new_stayselected = rent_own_new_stay[rent_own_new_stay.index.isin(selected_indexes)]

In [ ]:
rent_own_new_stayselected

,year,New Comers_rent,New Comers_own,stay_rent,stay_own
0,2015 Q1,0.0,0.0,318,579
1,2015 Q3,0.0,0.0,310,565
2,2016 Q1,0.0,0.0,310,565
3,2016 Q3,0.0,0.0,294,542
4,2017 Q1,11.0,14.0,282,520
5,2017 Q3,11.0,14.0,282,520
6,2018 Q1,31.0,53.0,278,517
7,2018 Q3,69.0,134.0,278,517
8,2019 Q1,90.0,195.0,271,493


In [ ]:
agents_stat_summary_by_yearæ

NameError: ignored

In [ ]:
fig

Their is a lot of ownown regrading how the population will look like because this is a complex matter

In [ ]:

agents_stat_summary_by_year['New Comers'] = agents_stat_summary_by_year['New Comers_income_low'] + agents_stat_summary_by_year['New Comers_income_medium'] +  + agents_stat_summary_by_year['New Comers_income_high']
agents_stat_summary_by_year['stay'] = agents_stat_summary_by_year['stay_income_low'] + agents_stat_summary_by_year['stay_income_medium'] +  + agents_stat_summary_by_year['stay_income_high']
agents_stat_summary_by_year['total_pop'] = agents_stat_summary_by_year['New Comers'] + agents_stat_summary_by_year['stay']
agents_stat_summary_by_year['New Comers_income_low_ratio'] = agents_stat_summary_by_year['New Comers_income_low']/agents_stat_summary_by_year['total_pop']
agents_stat_summary_by_year['New Comers_income_medium_ratio'] = agents_stat_summary_by_year['New Comers_income_medium']/agents_stat_summary_by_year['total_pop']
agents_stat_summary_by_year['New Comers_income_high_ratio'] = agents_stat_summary_by_year['New Comers_income_high']/agents_stat_summary_by_year['total_pop']

agents_stat_summary_by_year['stay_income_low_ratio'] = agents_stat_summary_by_year['stay_income_low']/agents_stat_summary_by_year['total_pop']
agents_stat_summary_by_year['stay_income_medium_ratio'] = agents_stat_summary_by_year['stay_income_medium']/agents_stat_summary_by_year['total_pop']
agents_stat_summary_by_year['stay_income_high_ratio'] = agents_stat_summary_by_year['stay_income_high']/agents_stat_summary_by_year['total_pop']


In [ ]:
#fig = px.line(agents_stat_summary_by_year, x="year"})
import plotly.graph_objects as go

fig = go.Figure()


fig.add_trace(go.Scatter(
    x=year_ranges,
    y=agents_stat_summary_by_year['New Comers_income_low_ratio'],
    legendgroup="New Comers",  # this can be any string, not just "group"
    legendgrouptitle_text="New Comers",
    name="Low Inocme",
    mode="lines",
    line=dict(color='royalblue', width=1)
))


fig.add_trace(go.Scatter(
    x=year_ranges,
    y=agents_stat_summary_by_year['New Comers_income_medium_ratio'],
    legendgroup="New Comers",  # this can be any string, not just "group"
    legendgrouptitle_text="New Comers",
    name="Medium Income",
    mode="lines",
    line=dict(color='royalblue', width=3)
))

fig.add_trace(go.Scatter(
    x=year_ranges,
    y=agents_stat_summary_by_year['New Comers_income_high_ratio'],
    legendgroup="New Comers",  # this can be any string, not just "group"
    legendgrouptitle_text="New Comers",
    name="High Income",
    mode="lines",
    line=dict(color='royalblue', width=5)
))


fig.add_trace(go.Scatter(
    x=year_ranges,
    y=agents_stat_summary_by_year['stay_income_low_ratio'],
    legendgroup="Staying",  # this can be any string, not just "group"
    legendgrouptitle_text="Staying",
    name="Renters",
    mode="lines",
    line=dict(color='firebrick', width=1)
))


fig.add_trace(go.Scatter(
    x=year_ranges,
    y=agents_stat_summary_by_year['stay_income_medium_ratio'],
    legendgroup="Staying",  # this can be any string, not just "group"
    legendgrouptitle_text="Staying",
    name="Medium",
    mode="lines",
    line=dict(color='firebrick', width=3)
))


fig.add_trace(go.Scatter(
    x=year_ranges,
    y=agents_stat_summary_by_year['stay_income_high_ratio'],
    legendgroup="Staying",  # this can be any string, not just "group"
    legendgrouptitle_text="Staying",
    name="High",
    mode="lines",
    line=dict(color='firebrick', width=5)
))


fig.update_layout(title="Income by class",template='plotly_white',yaxis = {'title' : "ratio"})

fig.show()

# set range

In [ ]:
#fig = px.line(agents_stat_summary_by_year, x="year"})
import plotly.graph_objects as go

fig = go.Figure()


fig.add_trace(go.Scatter(
    x=year_ranges,
    y=agents_stat_summary_by_year['New Comers'],
    legendgroup="New Comers",  # this can be any string, not just "group"
    legendgrouptitle_text="New Comers",
    name="New comers",
    mode="lines",
    line=dict(color='royalblue', width=1)
))


fig.add_trace(go.Scatter(
    x=year_ranges,
    y=agents_stat_summary_by_year['stay'],
    legendgroup="Staying",  # this can be any string, not just "group"
    legendgrouptitle_text="Staying",
    name="Staying",
    mode="lines",
    line=dict(color='firebrick', width=1)
))


fig.update_layout(title="Number of People",template='plotly_white',yaxis = {'title' : "Absolute"})

fig.show()

# set range

# Building Typology

In [ ]:
# Generate count status graph
def get_construction_typo_graph(construction_typo):
  fig = px.bar(bldg_status, x="status", y="count",width=500,height=200,template='plotly_white',title="Current Building Status",category_orders={'status':['Building before', 'Construction', 'Building after']})
  fig.update_layout(margin={"r":0,"t":35,"l":0,"b":0,"pad":0},
  showlegend=True,

  # font=dict(
  #     size=15,
  # )
  )
  fig.update_yaxes(automargin=True)
  fig.update_yaxes(range=[0, 70])
  return fig

In [ ]:
d1 = pd.to_datetime('2022-01-01')
d2 = d1 - datetime.timedelta(days=180)
bldgs1 = simulatedBldgsGdf[(simulatedBldgsGdf['start_date']< d1 ) & (simulatedBldgsGdf['end_date']> d1 )].copy().reset_index(drop=True)
construction_typo_v = bldgs1['project_ty'].value_counts().to_frame().reset_index()
construction_typo_v.rename(columns={'index':'project_ty','project_ty':'count'},inplace=True)
bldgs1 = simulatedBldgsGdf[(simulatedBldgsGdf['start_date']< d2 ) & (simulatedBldgsGdf['end_date']> d2 )].copy().reset_index(drop=True)
construction_typo_d = bldgs1['project_ty'].value_counts().to_frame().reset_index()
construction_typo_d.rename(columns={'index':'project_ty','project_ty':'count'},inplace=True)
#bldg_status_count = get_status_graph(bldg_status)

In [ ]:
construction_typo

NameError: ignored

In [ ]:
v_a = construction_typo[construction_typo['project_ty']==1]['count'].values[0]
v_r = construction_typo[construction_typo['project_ty']==2]['count'].values[0]
v_rr = construction_typo[construction_typo['project_ty']==2]['count'].values[0]

In [ ]:
d = pd.to_datetime('2025-01-01')
bldgs  = simulatedBldgsGdf[(simulatedBldgsGdf['start_date']< d ) & (simulatedBldgsGdf['end_date']> d )].copy().reset_index(drop=True)
bldgs1 = bldgs.copy()
agents_synced_buildings_to_date = pd.merge(agents_track_status,bldgs1[['project_nu','status','start_date']],left_on=['ProjNumber','bld_status'], right_on=['project_nu','status'])
agents_synced_buildings_to_date_stay_new = agents_synced_buildings_to_date[agents_synced_buildings_to_date['status_x']!='Leave'].drop_duplicates().reset_index(drop=True)

In [ ]:
agents_synced_buildings_to_date_stay_new['status_x']

0             stay
1             stay
2             stay
3             stay
4             stay
           ...    
1921    New Comers
1922    New Comers
1923    New Comers
1924    New Comers
1925    New Comers
Name: status_x, Length: 1926, dtype: object

In [ ]:
year_makrs = [year for year in np.arange(2015,2031,0.5)]
years_with_q2_makrs = []
for year in year_makrs:
    d = pd.to_datetime(f'{int(year)}-07-01')
    if year % 1 == 0:
        d = pd.to_datetime(f'{int(year)}-01-01')
    years_with_q2_makrs.append(d)

In [ ]:
age_grown1 = d.year-years_with_q2_makrs[0].year
agents_synced_buildings_to_date_stay_new.loc[agents_synced_buildings_to_date_stay_new['status_x']=='stay','age'] = agents_synced_buildings_to_date_stay_new.loc[agents_synced_buildings_to_date_stay_new['status_x']=='stay','age'] + age_grown1
agents_synced_buildings_to_date_stay_new.loc[agents_synced_buildings_to_date_stay_new['status_x']=='New Comers','age'] = agents_synced_buildings_to_date_stay_new.loc[agents_synced_buildings_to_date_stay_new['status_x']=='New Comers'].apply(lambda p: p['age'] + (d.year - p['start_date'].year),axis=1)

agents_synced_buildings_to_date_stay_new['age_group'] = pd.cut(agents_synced_buildings_to_date_stay_new['age'], [0,44,64,84,130],right=True, labels=["18-44", "45-64", "65-84", "85+"],ordered=True)
agents_stay_age_income =agents_synced_buildings_to_date_stay_new[['status_x','age_group','income_cat']].reset_index(drop=True).rename(columns={'status_x':'Stay or leave','age_group':'Age group','income_cat':'Income category'})

In [ ]:
agents_synced_buildings_to_date_stay_new[['status_x','age_group','income_cat']]

,status_x,age_group,income_cat
0,stay,45-64,Medium
1,stay,45-64,Medium
2,stay,45-64,Medium
3,stay,45-64,Medium
4,stay,45-64,Medium
...,...,...,...
1921,New Comers,45-64,High
1922,New Comers,45-64,High
1923,New Comers,45-64,High
1924,New Comers,85+,High


In [ ]:
fig = px.sunburst(agents_stay_age_income, path=['Stay or leave','Age group','Income category'])

In [ ]:
fig

In [ ]:
#https://github.com/Shai2u/demographic_estimation_dashboard_article/blob/main/assets/color_labels.xlsx
color_labels = pd.read_excel(
    'https://raw.githubusercontent.com/Shai2u/demographic_estimation_dashboard_article/main/assets/color_labels.xlsx')
colorDict = dict(zip(color_labels['label'], color_labels['colors_']))


In [ ]:
colorDict

{'18-44': 'hsv(205,80%,80%)',
 '45-64': 'hsv(205,60%,90%)',
 '65-84': 'hsv(205,30%,90%)',
 '85+': 'hsv(205,10%,90%)',
 'High': 'hsv(350,90%,75%)',
 'Low': 'hsv(350,25%,100%)',
 'Medium': 'hsv(350,50%,95%)',
 'New Comers': 'hsv(210,90%,50%)',
 'stay': 'hsv(350,90%,50%)'}

In [ ]:
d

Timestamp('2030-07-01 00:00:00')

In [ ]:
labels_text = fig.data[0].labels.tolist()

In [ ]:
labels_text

['Low',
 'Low',
 'Low',
 'Low',
 'Low',
 'Low',
 'Low',
 'Low',
 'Medium',
 'Medium',
 'Medium',
 'Medium',
 'Medium',
 'Medium',
 'Medium',
 'Medium',
 'High',
 'High',
 'High',
 'High',
 'High',
 'High',
 'High',
 'High',
 '18-44',
 '18-44',
 '45-64',
 '45-64',
 '65-84',
 '65-84',
 '85+',
 '85+',
 'New Comers',
 'stay']

In [ ]:
fig.data[0].marker.colors

# Sunburst

In [ ]:
    def sunburst_graph(r, year_, color_field='raw age', colorDict_=colorDict, group_color_dict=group_color_dict):
        title_ = f'{year_} : Market Vs Affordable Units Age/Income in WIRE'
        # color_discrete_map = color_group_map_
        colors_ = group_color_dict['colors_age'].unique().tolist()
        fig = px.sunburst(r, path=['ap_class', 'Age Group', 'Income'],
                          color=color_field, color_continuous_scale=colors_, title=title_,)

        labels_text = fig.data[0].labels.tolist()
        colorLabels = tuple(colorDict_[item] for item in labels_text)
        fig.data[0].marker.colors = colorLabels
        fig.update_traces(textinfo="label+percent entry")
        fig.update_layout(showlegend=False, margin=dict(l=50, r=50, t=100, b=50), legend=dict(
            yanchor="top", y=1, xanchor="left", x=1, orientation="h"), width=sim_plot.cont_width, height=sim_plot.tl_height+50,font=dict(size=sim_plot.textSize_))
        return fig

NameError: ignored

# Bubble Figure

In [21]:
    def bubbleAgeIncomeClass(r, year_, titleText_):
        r2 = r.groupby(['ap_class', 'group_name', 'Age Group', 'Income']).agg(
            {'raw age': 'count'}).reset_index().rename(columns={'raw age': 'count', 'ap_class': 'Ap Type'})

        title_ = str(year_)+' '+titleText_ + ' Age/Income'

        fig = px.scatter(r2, x="Age Group", y="Income",
                         size="count", color="group_name", color_discrete_map=colorDictMerge, facet_col='Ap Type', title=title_, size_max=30,
                         category_orders={"Age Group": ["18-44", "45-64", "65-84", "85+"],
                                          "Income": ['Upper', 'Middle', 'Moderate', 'Low']}, template='ggplot2')
        fig.update_layout(showlegend=False, margin=dict(l=50, r=50, t=100, b=50), width=sim_plot.cont_width, height=sim_plot.tl_height,font=dict(size=sim_plot.textSize_))

        return fig

In [23]:
fig = make_subplots(rows=1, cols=2,subplot_titles=("Reference Date", "Current Date"))

fig.add_trace(
    go.Scatter(x=[1, 2, 3], y=[4, 5, 6], mode="markers",name='Reference Date',marker=dict(size=20,color='green')),
    row=1, col=1)

fig.add_trace(
    go.Scatter(x=[20, 30, 40], y=[50, 60, 70], mode="markers",name='Current Date'),
    row=1, col=2
)


fig.update_layout(height=600, width=800, title_text="Side By Side Subplots", margin=dict(l=0, r=0, t=50, b=0),legend=dict(orientation="h", yanchor="top",y=0.99,xanchor="left",x=0.01),template='plotly_white')
fig.show()

[18, 32, 0, 31, 71, 0, 14, 29, 0, 0, 0, 0]

In [ ]:

#hovertext

# fig = go.Figure(go.Scatter(
#     x = [1,2,3,4,5],
#     y = [2.02825,1.63728,6.83839,4.8485,4.73463],
#     hovertemplate =
#     '<i>Price</i>: $%{y:.2f}'+
#     '<br><b>X</b>: %{x}<br>'+
#     '<b>%{text}</b>',
#     text = ['Custom text {}'.format(i + 1) for i in range(5)],
#     showlegend = False))

# fig.add_trace(go.Scatter(
#     x = [1,2,3,4,5],
#     y = [3.02825,2.63728,4.83839,3.8485,1.73463],
#     hovertemplate = 'Price: %{y:$.2f}<extra></extra>',
#     showlegend = False))

# fig.update_layout(
#     hoverlabel_align = 'right',
#     title = "Set hover text with hovertemplate")

# fig.show()

In [ ]:
d

Timestamp('2022-01-01 00:00:00')